# Author: Umang Srivastav

Load the save model from the exeuted file "RN-50 and some test on variations" or the one I have kept in the folder. Then execute the FIrst 18 layers + a FC layer modified model of RN50.

In [ ]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models

model=models.resnet50(pretrained=False, num_classes=100)
model.load_state_dict(torch.load('model50.pt'))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class ModifiedResNet50(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        
        resnet=model#.resnet
        self.conv1=resnet.conv1
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool
        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        del self.layer2[3]
        # del self.layer2[2].conv2
        # del self.layer2[2].conv3
        # del self.layer2[2].bn2
        # del self.layer2[2].bn3
        self.avgpool = resnet.avgpool
        self.fc=torch.nn.Linear(in_features=512, out_features=100, bias=True)

        # self.fc2=torch.nn.Linear(in_features=100, out_features=500, bias=True)
        # self.fc3=torch.nn.Linear(in_features=500, out_features=100, bias=True)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        # x = self.fc2(x)
        # x = self.fc3(x)
        
        return x


model = ModifiedResNet50(model) 
model=model.to(device)
print(model)
import torchvision
for name, param in model.named_parameters():
    if name.startswith('fc'):
        param.requires_grad = True
    else:
        param.requires_grad = False

# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(name, "is trainable")
#     else:
#         print(name, "is not trainable")

transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071, 0.4865, 0.4409],
                         std=[0.2673, 0.2564, 0.2762])
])


trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)


validset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
validloader = torch.utils.data.DataLoader(validset, batch_size=32, shuffle=False, num_workers=2)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

num_epochs = 100
for epoch in range(num_epochs):
    
    for images, labels in trainloader:
        
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()

        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in validloader:
            if torch.cuda.is_available():
                images = images.cuda()
                labels = labels.cuda()

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Epoch [{epoch + 1}/{num_epochs}], Test Accuracy: {accuracy:.2f}%")

torch.save(model.state_dict(), 'Modmodel50-18.pt')

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ModifiedResNet50(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), s

100%|██████████| 169001437/169001437 [00:03<00:00, 52752954.63it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified
Epoch [1/100], Test Accuracy: 29.69%
Epoch [2/100], Test Accuracy: 32.76%
Epoch [3/100], Test Accuracy: 35.17%
Epoch [4/100], Test Accuracy: 35.10%
Epoch [5/100], Test Accuracy: 34.43%
Epoch [6/100], Test Accuracy: 35.95%
Epoch [7/100], Test Accuracy: 37.12%
Epoch [8/100], Test Accuracy: 38.17%
Epoch [9/100], Test Accuracy: 36.50%
Epoch [10/100], Test Accuracy: 38.27%
Epoch [11/100], Test Accuracy: 36.96%
Epoch [12/100], Test Accuracy: 36.63%
Epoch [13/100], Test Accuracy: 39.07%
Epoch [14/100], Test Accuracy: 38.93%
Epoch [15/100], Test Accuracy: 38.79%
Epoch [16/100], Test Accuracy: 37.82%
Epoch [17/100], Test Accuracy: 37.67%
Epoch [18/100], Test Accuracy: 39.64%
Epoch [19/100], Test Accuracy: 39.79%
Epoch [20/100], Test Accuracy: 40.48%
Epoch [21/100], Test Accuracy: 38.50%
Epoch [22/100], Test Accuracy: 38.35%
Epoch [23/100], Test Accuracy: 40.06%
Epoch [24/100], Test Accuracy: 39.14%
Epoch [